## Problemas de Transporte

### Problema #1

### Formulación del problema

#### Variables de decisión:

Sea x_ij la cantidad de **millones de galones** que se envían desde la **refinería \( i \)** al **área de distribución \( j \)**, con:

$$
( i \in \{1, 2, 3\} )  (Refinerías)
$$
$$
( j \in \{1, 2, 3\} )  (Áreas de distribución)
$$

#### Parámetros:

**Oferta de gasolina diaria (millones de galones):**
- Refinería 1: s_1 = 6 
- Refinería 2: s_2 = 5 
- Refinería 3: s_3 = 8 

**Demanda diaria por área (millones de galones):**
- Área 1: d_1 = 4 
- Área 2: d_2 = 8 
- Área 3: d_3 = 7

**Distancia entre refinerías y áreas (en km):**

|              | Área 1 | Área 2 | Área 3 |
|--------------|--------|--------|--------|
| Refinería 1  | 120    | 180    | —      |
| Refinería 2  | 300    | 100    | 80     |
| Refinería 3  | 200    | 250    | 120    |

**Costo de transporte (USD por millón de galones):**

El costo de transporte es **$0.10** por cada 1000 galones por km, es decir:

$$
\text{Costo}_{ij} = 0.10 \times \text{Distancia}_{ij}
$$

Entonces la matriz de costos c_ij queda:

|              | Área 1 | Área 2 | Área 3 |
|--------------|--------|--------|--------|
| Refinería 1  | 12     | 18     | —      |
| Refinería 2  | 30     | 10     | 8      |
| Refinería 3  | 20     | 25     | 12     |


#### Función Objetivo:

Minimizar el costo total de transporte:

$$
\min Z = \sum_{i=1}^{3} \sum_{j=1}^{3} c_{ij} \cdot x_{ij}
$$

(sin incluir rutas prohibidas, como x_13.)


#### Restricciones:

**Oferta por refinería:**

$$
\begin{aligned}
x_{11} + x_{12} &\leq 6 \quad \text{(Refinería 1)} \\
x_{21} + x_{22} + x_{23} &\leq 5 \quad \text{(Refinería 2)} \\
x_{31} + x_{32} + x_{33} &\leq 8 \quad \text{(Refinería 3)} \\
\end{aligned}
$$

**Demanda por área de distribución:**

$$
\begin{aligned}
x_{11} + x_{21} + x_{31} &= 4 \quad \text{(Área 1)} \\
x_{12} + x_{22} + x_{32} &= 8 \quad \text{(Área 2)} \\
x_{23} + x_{33} &= 7 \quad \text{(Área 3)} \\
\end{aligned}
$$

**Restricciones de no negatividad:**

$$
x_{ij} \geq 0 \quad \forall i,j
$$

**Restricción de conectividad:**

$$
x_{13} = 0 \quad \text{(Refinería 1 no puede enviar a Área 3)}
$$

### Solución

In [11]:
using JuMP
using Ipopt 
using HiGHS 
using Optimization

In [12]:
model = Model()

A JuMP Model
├ solver: none
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [14]:
# Refinerías (oferta)
supply = [6.0, 5.0, 8.0]  # millones de galones

# Áreas de distribución (demanda)
demand = [4.0, 8.0, 7.0]  # millones de galones

# Costos de transporte en USD por millón de galones
costos = [
    12.0  18.0   1e6;    # Refinería 1 
    30.0  10.0   8.0;    # Refinería 2
    20.0  25.0  12.0     # Refinería 3
]

3×3 Matrix{Float64}:
 12.0  18.0   1.0e6
 30.0  10.0   8.0
 20.0  25.0  12.0

In [15]:
# variables de decisión
@variable(model, x[1:3, 1:3] >= 0)  # x[i,j] = cantidad enviada de refinería i a área j

# restricciones de oferta
for i in 1:3
    @constraint(model, sum(x[i, j] for j in 1:3) <= supply[i])
end

# restricciones de demanda
for j in 1:3
    @constraint(model, sum(x[i, j] for i in 1:3) == demand[j])
end

# función objetivo: minimizar el costo total de transporte
@objective(model, Min, sum(costos[i,j] * x[i,j] for i in 1:3, j in 1:3))


12 x[1,1] + 18 x[1,2] + 1000000 x[1,3] + 30 x[2,1] + 10 x[2,2] + 8 x[2,3] + 20 x[3,1] + 25 x[3,2] + 12 x[3,3]

In [17]:
set_optimizer(model, HiGHS.Optimizer)

In [18]:
optimize!(model)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 6 rows; 9 cols; 18 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [8e+00, 1e+06]
  Bound  [0e+00, 0e+00]
  RHS    [4e+00, 8e+00]
Presolving model
6 rows, 9 cols, 18 nonzeros  0s
Dependent equations search running on 3 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
6 rows, 9 cols, 18 nonzeros  0s
Presolve : Reductions: rows 6(-0); columns 9(-0); elements 18(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(19) 0s
          6     2.4300000000e+02 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 6
Objective value     :  2.4300000000e+02
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.01


In [19]:
println("Solución óptima:")
for i in 1:3, j in 1:3
    println("x[$i,$j] = ", value(x[i,j]))
end

println("Costo total mínimo: \$", objective_value(model))

Solución óptima:
x[1,1] = 4.0
x[1,2] = 2.0
x[1,3] = 0.0
x[2,1] = 0.0
x[2,2] = 5.0
x[2,3] = 0.0
x[3,1] = 0.0
x[3,2] = 1.0
x[3,3] = 7.0
Costo total mínimo: $243.0


<small>

### Análisis de Resultados 

Luego de resolver el modelo de transporte con JuMP, se obtuvo la siguiente solución óptima:

#### Distribución óptima de envíos (en millones de galones):

| Refinería → Área | Envío |
|------------------|--------|
| Refinería 1 → Área 1 | 4.0 |
| Refinería 1 → Área 2 | 2.0 |
| Refinería 2 → Área 2 | 5.0 |
| Refinería 3 → Área 2 | 1.0 |
| Refinería 3 → Área 3 | 7.0 |


- Todas las demás variables tienen valor 0, incluyendo la ruta prohibida (Refinería 1 → Área 3).

#### Verificación de restricciones:

- **Oferta utilizada:**
  - Refinería 1: 4 + 2 = 6 ≤ 6 
  - Refinería 2: 5 ≤ 5 
  - Refinería 3: 1 + 7 = 8 ≤ 8 

- **Demanda satisfecha:**
  - Área 1: 4 
  - Área 2: 2 + 5 + 1 = 8 
  - Área 3: 7 

**El modelo cumple con todas las restricciones de capacidad y demanda.**


### Costo total mínimo:

$$
Z = (4 \cdot 12) + (2 \cdot 18) + (5 \cdot 10) + (1 \cdot 25) + (7 \cdot 12) = \$243
$$


### Observaciones:

- La Refinería 1 cubre completamente el Área 1 y parcialmente el Área 2.
- La Refinería 2 solo contribuye al Área 2, usando toda su capacidad.
- La Refinería 3 se encarga completamente del Área 3 y apoya al Área 2 con 1 millón.
- No se usa la ruta Refinería 1 → Área 3 debido a su **costo prohibitivo**.
- Se respetaron los límites de capacidad y se cumplió exactamente la demanda de cada área.


**Conclusión:**  
La solución es óptima, económica **$243** 
y cumple con todas las condiciones del problema de transporte balanceado $$ \sum \text{oferta} = \sum \text{demanda} $$

</small>

### Problema #3

### Formulación del Modelo

* Índices

$$
i = 1,2,3,4\quad (\text{trabajadores}), 
\quad j = 1,2,3,4\quad (\text{puestos}).
$$

* Parámetros

$$
c_{ij} = 
\begin{pmatrix}
50 & 50 & - & 20\\
70 & 40 & 20 & 30\\
90 & 30 & 50 & -\\
70 & 20 & 60 & 70
\end{pmatrix},
$$

donde $c_{1,3}=+\infty$ (trabajador 1 no puede ir al puesto 3) y
$c_{3,4}=+\infty$ (trabajador 3 no puede ir al puesto 4).

* Variables de decisión

$$
x_{ij} =
\begin{cases}
1 & \text{si al trabajador }i\text{ se le asigna el puesto }j,\\
0 & \text{en otro caso.}
\end{cases}
$$

* Función objetivo (minimizar coste total)

$$
\min \sum_{i=1}^4\sum_{j=1}^4 c_{ij}\,x_{ij}.
$$

* Restricciones

1. Cada trabajador hace exactamente un puesto:
   $\displaystyle\sum_{j=1}^4 x_{ij} = 1\quad \forall\,i=1,\dots,4.$
2. Cada puesto es cubierto por exactamente un trabajador:
   $\displaystyle\sum_{i=1}^4 x_{ij} = 1\quad \forall\,j=1,\dots,4.$
3. $x_{ij}\in\{0,1\}$.

---

### Solución

In [ ]:
import JuMP
import HiGHS
import Ipopt
import Optimization

In [10]:
# Costos
costs_p3 = [
    50 50 1e6 20;
    70 40 20 30;
    90 30 50 1e6;
    70 20 60 70
]

4×4 Matrix{Float64}:
 50.0  50.0   1.0e6  20.0
 70.0  40.0  20.0    30.0
 90.0  30.0  50.0     1.0e6
 70.0  20.0  60.0    70.0

In [12]:
# Modelo
model = Model(HiGHS.Optimizer)

# Variables 
@variable(model, x[1:4, 1:4], Bin)

# Objetivo
@objective(model, Min, sum(costs_p3[i, j] * x[i, j] for i in 1:4, j in 1:4))

# Restricciones

# Cada trabajador i asignado a un único puesto
@constraint(model,  [i in 1:4], sum(x[i, j] for j in 1:4) == 1)

# Cada puesto j ocupado por un único trabajador
@constraint(model,  [j in 1:4], sum(x[i, j] for i in 1:4) == 1)

4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 x[1,1] + x[2,1] + x[3,1] + x[4,1] == 1
 x[1,2] + x[2,2] + x[3,2] + x[4,2] == 1
 x[1,3] + x[2,3] + x[3,3] + x[4,3] == 1
 x[1,4] + x[2,4] + x[3,4] + x[4,4] == 1

In [13]:
# Resolver
optimize!(model)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 8 rows; 16 cols; 32 nonzeros; 16 integer variables (16 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e+01, 1e+06]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
8 rows, 16 cols, 32 nonzeros  0s
8 rows, 16 cols, 32 nonzeros  0s
Objective function is integral with scale 0.1

Solving MIP model with:
   8 rows
   16 cols (16 binary, 0 integer, 0 implied int., 0 continuous, 0 domain fixed)
   32 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraint

In [14]:
# Resultados
println("Coste óptimo: \$", objective_value(model))
for i in 1:4, j in 1:4
    if value(x[i, j]) > 0.5
        println("Trabajador $i -> Puesto $j (coste=", costs_p3[i, j], ")")
    end
end

Coste óptimo: $140.0
Trabajador 1 -> Puesto 4 (coste=20.0)
Trabajador 2 -> Puesto 3 (coste=20.0)
Trabajador 3 -> Puesto 2 (coste=30.0)
Trabajador 4 -> Puesto 1 (coste=70.0)
